## COM5507 Project Work_Scraping
- Group Member: He Weiyuan, Chen Dantong, Cai Yanhuan, Zhang Chuhan
- Targeted website: https://www.zhipin.com
- Targeted job: Data Analyst, Media Operation
- Targeted city: First-tier city(including Beijing, Shanghai, Guangzhou, Shenzhen)

In [16]:
import requests 
import re
from bs4 import BeautifulSoup
from lxml import etree  # 使用xpath语法解析
import pandas as pd
from time import sleep, time
import random
import csv

In [22]:
# create url we want to scrape
def url_create():
    headers = {
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:9527.0) Gecko/20100101 Firefox/9527.0"}

    city_list = ['杭州']
    position_list=['运营']
    for city in city_list:
        print("I am in {}".format(city))
        for position in position_list:
            url = "https://www.zhipin.com/job_detail/?query={}&scity=101020100&page=".format(position)
            #print(url)
            page=4
            for n in range(1,4):
                url_page = url+str(page)
                page += 1
                print(url_page)
                time_sleep = random.randrange(5, 8)
                #sleep(time_sleep*10)
                re=requests.get(url_page,headers=headers)
                re.encoding = 'utf-8'
                soup = BeautifulSoup(re.text,'html.parser')
                detail_url(soup)
            print("{} link is done".format(city))

In [23]:
def detail_url(data):
    for item in data.find_all('div','job-primary'):
        for info_primary in item.find_all('div','info-primary'): 
            for link in info_primary.find_all('a'):
                urls.append('https://www.zhipin.com/'+link.get('href'))
    print(urls)

In [24]:
headers = { 'user-agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1) Gecko/20070309 Firefox/2.0.0.3'}
regx_obj = re.compile(r'<br/>|<(em).*?>.*?</\1>')

urls = []
def send_request(url_path, headers):    
    response = requests.get(url=url_path, headers=headers)
    response = regx_obj.sub('', response.text)
    return response

def parse_data():
    url_create()
    for url in urls:
        html = send_request(url, headers)
        html_obj = etree.HTML(html)
        item = {}
        # 发布日期
        item['publishTime'] = html_obj.xpath(".//div[@class='info-primary']//span[@class='time']/text()")[0]
        # 职位名
        item['position'] = html_obj.xpath(".//div[@class='info-primary']//h1/text()")[0]
        # 薪水
        item['salary'] = html_obj.xpath(".//div[@class='info-primary']//span[@class='badge']/text()")[0].strip()
        # 公司名称
        item['companyName'] = html_obj.xpath("//div[@class='info-company']//h3/a/text()")[0]
        # 公司类型
        item['companyType'] = html_obj.xpath("//div[@class='info-company']//p//a/text()")[0]
        # 公司规模
        item['companySize'] = html_obj.xpath("//div[@class='info-company']//p/text()")[0]
        # 工作职责
        item['responsibility'] = html_obj.xpath("//div[@class='job-sec']//div[@class='text']/text()")[0].strip()
        # 招聘要求
        item['requirement'] = html_obj.xpath("//div[@class='job-banner']//div[@class='info-primary']//p/text()")[0]
        print(item)
        result_save(item)  
        time_sleep = random.randrange(5, 8)
        sleep(time_sleep)
    #print(len(item))

In [25]:
#结果信息存储
def result_save(result_item):
    with open('data_operation_detail_hz.csv','a',newline='',encoding='utf-8') as csvfile:   #打开一个csv文件，用于存储
        fieldnames=['publishTime','position','salary','companyName','companyType','companySize','responsibility','requirement']
        writer=csv.DictWriter(csvfile,fieldnames=fieldnames)
        writer.writerow(result_item)

#单行信息存储
def save_single_info(info):
    with open('data_operation_detail_hz.csv','a',newline='',encoding='utf-8') as csvfile:
        writer=csv.writer(csvfile)
        if type(info) == list:
            writer.writerow(info)
        else:
            writer.writerow([info])


#主程序
def main():
    box_header = ['publishTime','position','salary','companyName','companyType','companySize','responsibility','requirement']
    save_single_info(box_header)  #写入表头
    parse_data()

In [26]:
#运行程序
if __name__ == '__main__':
    start_time = time()
    print("working...")
    main()
    end_time = time()
    print("运行结束，用时：")
    total_time = (end_time - start_time)/60
    print(total_time)

working...
I am in 杭州
https://www.zhipin.com/job_detail/?query=运营&scity=101020100&page=4
['https://www.zhipin.com//job_detail/775b339c72fc9c291nx_0967Fls~.html', 'https://www.zhipin.com//job_detail/baa33e2beadd33351X153dm6FlQ~.html', 'https://www.zhipin.com//job_detail/ad53ddf4099a22fa1HR93d6_FFM~.html', 'https://www.zhipin.com//job_detail/87886f704907079c1HV72dy5FFM~.html', 'https://www.zhipin.com//job_detail/6d8e098d5dfd34b61X1z2dS6GVM~.html', 'https://www.zhipin.com//job_detail/ee6e45e840c2acf01Xx82tu5EVA~.html', 'https://www.zhipin.com//job_detail/de6b63a2c283d18f1XZ6092_F1A~.html', 'https://www.zhipin.com//job_detail/e4b356237e25b7c31HR90ti7GFQ~.html', 'https://www.zhipin.com//job_detail/953f1007087c72391HR_39u9GVU~.html', 'https://www.zhipin.com//job_detail/d5e6b194e59e76f41XJ63t6-ElU~.html', 'https://www.zhipin.com//job_detail/4d327d332fce46581HR90ti1F1M~.html', 'https://www.zhipin.com//job_detail/082bb4849e862dc51X180t60FVI~.html', 'https://www.zhipin.com//job_detail/d8785e858d

{'publishTime': '发布于2018-10-24 10:53', 'position': '社群运营', 'salary': '6K-9K', 'companyName': '英语流利说', 'companyType': '移动互联网', 'companySize': '已上市1000-9999人', 'responsibility': '岗位职责：1、.指导用户正确地使用app，并给与学习上的指导（见招拆招你来我往，和学员共同进步～）2、日常维护社群用户，确保用户的活跃度（治疗拖延症哪家强？我们期待干劲十足的你！）3、定期做数据分析，调整运营策略（你也有千军万马噢～）4、提升用户满意度，协助完成团队的指标（与一个有爱的团队一同工作～）5、完成其它需要协助的工作我们希望你：1、学历大专或以上，口语流利2、具备较强的社群活动策划、维护和执行能力3、热爱语言学习和帮助他人学语言，具有比较强服务意识4、团队意识较强，具有责任感和较强的学习能力5、能够熟练使用相关办公软件', 'requirement': '城市：上海经验：经验不限学历：大专'}
{'publishTime': '发布于2018-11-08 15:50', 'position': '游戏运营', 'salary': '10K-20K', 'companyName': '米哈游', 'companyType': '游戏', 'companySize': '不需要融资500-999人', 'responsibility': '岗位职责：1. 负责游戏产品日常运营活动的策划、宣传和实施，跟进运营活动的效果； 2. 负责运营相关数据的监护，分析运营活动的相关数据，根据需要调整活动策略； 3. 熟悉游戏运营的各种情况，及时处理游戏的日常运营事件； 4. 协调日常运营工作和资源，推进运营活动的不断优化； 5. 结合运营、市场等因素调整整体运营节奏，协助完成各类线上线下活动。 任职要求： 1. 具有游戏产品活动运营经验，在运营活动方面有一定积累； 2. 热爱互联网行业，忠实的游戏玩家，熟悉多种手机游戏和网络游戏； 3. 具有优秀的团队合作能力，拥有良好的创新精神，工作认真负责。', 'requirement': '城市：上海经验：经验不限学历：不限'}
{'publishTime': '发布于2018-12-

{'publishTime': '发布于2018-11-27 09:15', 'position': '直播运营', 'salary': '10K-20K', 'companyName': '上海金大师', 'companyType': '互联网金融', 'companySize': '已上市1000-9999人', 'responsibility': '职位描述：1.负责主播、公会、经纪公司的资源拓展、入驻对接，丰富直播样式；2.负责主播的日常管理，与公会、经纪公司关系的维护，形成良好的合作关系；3.制定能有效提升主播开播率、开播时长等核心数据的运营方案，制定主播等级运营策略，并推动落地；4.定期收集主播的建议并生成文档反馈给产品经理，并推动合理建立的落地；5. 能根据目标拆分项目执行计划，对房间活跃、营收指标负责；任职资格:岗位要求：1. 了解直播行业，有公会管理、主播对接的运营经验；2. 对语音直播有较深入的认知，并对语音、电台内容有较高的兴趣；3. 沟通能力强，有优秀的谈判技巧和团队协作能力；4. 熟悉直播和公会生态，了解主播和付费用户的核心诉求；5.一定的圈内资源和人脉，并能很好整合资源，商务谈判和执行力强，具有开拓精神。', 'requirement': '城市：上海经验：1-3年学历：本科'}
{'publishTime': '发布于2018-12-04 17:28', 'position': '运营经理', 'salary': '20K-25K', 'companyName': '好未来', 'companyType': '互联网', 'companySize': '已上市10000人以上', 'responsibility': '岗位职责1.负责智慧学校项目中公立校的运营工作，包括：需求收集、进度反馈、项目进展的跟进和内外部资源协调2.负责跟踪产品落地后的关键KPI，保证项目效果任职资格：1.5年+客户运营、项目交付、项目管理相关工作经验，本科及以上学历2.具备良好的沟通技巧3.具备教育类学校运营相关经验优先', 'requirement': '城市：上海经验：3-5年学历：本科'}
{'publishTime': '发布于2018-10-15 20:16', 'position': '运营助理', 'salary': '4K-6K', 'comp

{'publishTime': '发布于2018-12-10 17:12', 'position': '运营助理', 'salary': '5K-10K', 'companyName': '橙絮网络', 'companyType': '互联网', 'companySize': '不需要融资0-20人', 'responsibility': '职位描述：1、负责微信公众号产品的调整，发送实时推文；2、熟悉产品UV转化，跟踪推广效果，汇总、分析数据并反馈；3、深度挖掘用户需求，提高产品收益。任职要求：1、有互联网信息流广告投放运营经验的优先；2、有微信公众号运营的相关经验者优先；3、具有积极的工作心态，与良好的沟通能力；4、工作细致耐心，具备良好团队精神。', 'requirement': '城市：上海经验：经验不限学历：大专'}
{'publishTime': '发布于2018-07-09 11:25', 'position': '产品运营', 'salary': '10K-20K', 'companyName': '携程旅行网', 'companyType': '互联网', 'companySize': '已上市10000人以上', 'responsibility': '岗位描述：1.\t参与全球化POI平台的建立，负责信息挖掘和POI维护；2.\t通过日常反馈和数据分析，挖掘POI信息质量对用户体验的影响，协助产品/技术落地解决方案；3.\t负责与业务方合作对接，针对业务需求进行合理评估，产品化手段提升需求处理的效率和质量；4.\t评估解决策略的风险和收益，制定并严格遵守数据处理的标准规范和流程，提升存量POI质量；5.\t运营跟进已成型的业务流程，对执行结果进行监控分析，在日常运营中持续迭代优化职位要求：1. 本科及以上学历，2年以上产品运营工作经验，拥有数据运营项目管理经验，熟练掌握日常工具的excel、ppt，会SQL加分；2. 独立思考，对用户体验敏感，善于在日常运营中发现既有流程的问题，并协调资源进行改善；3. 善于沟通，良好的团队协作能力，结果导向；4. 有OTA经验或对旅游行业有较深入的了解和认知者优先', 'requirement': '城市：上海经验：1-3年学历：本科'}
{'publishTime': '发布于2018-09-17 12:06

{'publishTime': '发布于2018-11-27 11:09', 'position': '产品运营', 'salary': '10K-20K', 'companyName': '上海金大师', 'companyType': '互联网金融', 'companySize': '已上市1000-9999人', 'responsibility': '你能得到什么：1.互联网短视频行业的绝佳入场机会，成为炙手可热的行业人才2.美国纳斯达克上市集团，充足的市场与资金资源3.业界一流领先的产品研发团队，没有做不到，只有想不到你运营的产品：一款股票类视频拍摄、制作、分发、营销、统计分析、粉丝管理的移动端工具。苹果应用市场：搜索“股拍”安卓下载地址：https://fir.im/gpzspro你运营的对象：券商、投顾公司、证券自媒体、财经自媒体你的职责：1.运营一款口碑绝佳的股市短视频拍摄 APP “股拍”2.维护“股拍”粉丝群，并通过电话、在线IM与用户沟通交流，帮助用户拍出优质的视频4.分析推广数据，挖掘用户需求，与产品团队沟通，为研发团队提供建议5.参与线下产品推介活动的协调与执行6.提升“股拍”产品在用户心中的影响力我们期望的你是这样的：1.了解券商或投顾公司的业务模式、有炒股经验，会经常看股市相关的分析报告及资讯内容2.有较好的客户服务意识，良好的沟通倾听能力3.有良好的学习能力，喜欢阅读书籍或收听“得到”类的知识APP通过帮助全行业证券内容生产方提升营销、服务效能，让你的自我价值实现最大化！', 'requirement': '城市：上海经验：1-3年学历：大专'}
{'publishTime': '发布于2018-12-11 10:03', 'position': '用户运营', 'salary': '6K-8K', 'companyName': '艺星美容医院', 'companyType': '医疗/护理/卫生', 'companySize': '不需要融资500-999人', 'responsibility': '工作要求：1. 负责平台线上客户资源的拓展及维护，分析客户需求，与客户谈判并达成合作，评估资源的可合作性，制定可复制营销方案；\xa02. 负责客户关系的跟进及维护，负责解决与客户合作过程中遇到的问题，积极的提供解决方案；3. 具有良好的学

{'publishTime': '发布于2018-12-06 13:53', 'position': '客户运营', 'salary': '6K-8K', 'companyName': '百望股份', 'companyType': '互联网', 'companySize': '不需要融资500-999人', 'responsibility': '岗位职责（1）运营中通生态。通过电话、邮件、微信等在线工具及渠道与客户进行沟通，提供上线指引、解答用户的业务咨询问题；（2）根据客户需求，提供专业的解决方案，提高客户满意度，并承担部分需求、商务的工作；（3）按照客服人员工作要求，及时接听公司客服电话或线上咨询服务解答并做好记录，了解客户需求，主动帮助客户解决问题；（4）提供系统升级验证支持，保障系统正常运行；（5）参与培训用户，提供现场指导等售后技术支持工作。任职要求有客服从业、税务背景经验者优先考虑；有客户运营经验的优先考虑。专业知识/技能/能力(1)做过运维支持或售后技术支持类似性工作；(2)良好的客户沟通能力；(3)具有良好的客户形象，口齿清晰；(4)学习能力强，极强的执行力和沟通能力，具备良好的服务意识；(5)计算机基础软件(word ， windows）操作熟练，打字熟练。', 'requirement': '城市：上海经验：1-3年学历：大专'}
{'publishTime': '发布于2018-09-25 16:13', 'position': '社区运营', 'salary': '10K-15K', 'companyName': '海天医药', 'companyType': '制药', 'companySize': '不需要融资1000-9999人', 'responsibility': '工作职责：1、负责APP平台问答专区的日常维护及内容运营。2、创建平台使用及与平台运营内容相关的常见Q&A问答。3、创建高频热门、置顶、活跃互动话题。4、建立平台对用户提问的快速回复机制及标准回复模板。5、筛选、把控问答专区发布信息的内容质量及内容相关度。6、对问答专区相关数据（问题数据、用户数据等）进行分析总结。7、对问答专区用户体验进行持续协助。任职资格：1、本科及本科以上学历，广告、传媒、传播学、市场营销相关专业优先；2、具有2年以上用户运营经验；3、熟悉互联网社区

IndexError: list index out of range